Code from https://github.com/iamtrask/Grokking-Deep-Learning/blob/master/Chapter12%20-%20Intro%20to%20Recurrence%20-%20Predicting%20the%20Next%20Word.ipynb

Download & Preprocess the IMDB Dataset

In [7]:
# Download reviews.txt and labels.txt from here: https://github.com/udacity/deep-learning/tree/master/sentiment-network

def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('data/reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('data/labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()


# Preprocess dataset:

import sys

f = open('data/reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('data/labels.txt')
raw_labels = f.readlines()
f.close()

# Split reviews and get all words. Tokens is a list of reviews, but each review is a list of words
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

# Get distinct words
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

# Assign an integer to each word
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

#input_dataset is a list of reviews but each word is replaced with the corresponding index
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

#labels
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

Train a NN to predict reviews

In [11]:
import numpy as np 
np.random.seed(1) 

def sigmoid(x): 
    return 1/(1 + np.exp(-x)) 

alpha, iterations = (0.01, 2) 
hidden_size = 100 
weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1 
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1 
correct,total = (0,0) 

for iter in range(iterations): 
    # train on first 24,000 
    for i in range(len(input_dataset)-1000): 
        x,y = (input_dataset[i],target_dataset[i]) 
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid 
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax  #Stavros: sigmoid
        
        layer_2_delta = layer_2 - y # compare pred with truth

        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop 
        
        weights_0_1[x] -= layer_1_delta * alpha 
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha 
        
        if(np.abs(layer_2_delta) < 0.5): 
            correct += 1 
        total += 1 
        
        if(i % 10 == 9): 
            progress = str(i/float(len(input_dataset))) 
            sys.stdout.write('\rIter:'+str(iter) +
                             ' Progress:' + progress[2:4] +
                             '.' +progress[4:6] +
                             '% Training Accuracy:' +
                             str(correct/float(total)) + '%') 
        
correct,total = (0,0) 
for i in range(len(input_dataset)-1000,len(input_dataset)): 
    x = input_dataset[i] 
    y = target_dataset[i] 
    
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) 
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) 
    
    if(np.abs(layer_2 - y) < 0.5): 
        correct += 1 
        
    total += 1 
    
print("Test Accuracy:" + str(correct / float(total)))

Iter:1 Progress:95.99% Training Accuracy:0.8670208333333334%Test Accuracy:0.851


The Surprising Power of Averaged Word Vectors

In [129]:
import numpy as np
from collections import Counter

#Original
#norms = np.sum(weights_0_1 * weights_0_1,axis=1)
#norms.resize(norms.shape[0],1)
#normed_weights = weights_0_1 * norms

#Stavros
#L1 normalization
#norms = np.sum(weights_0_1 ** 2 ,axis=1, keepdims=1)
#normed_weights = weights_0_1 / norms

#L2 normalization
norms = np.sqrt(np.sum(weights_0_1 ** 2 ,axis=1, keepdims=1))
normed_weights = weights_0_1 / norms


def make_sent_vect(words):
    #filter(lambda x:x in word2index,words) -> Get only words that exist in word2index
    #map(lambda x:word2index[x] -> get index of each word
    indices = list(map(lambda x:word2index[x],filter(lambda x:x in word2index,words)))
    #take the embeddings of each word and find the mean (normalized)
    return np.mean(normed_weights[indices],axis=0)

#Create an array that has the average embeddings for the words of each review
reviews2vectors = list()
for review in tokens: # tokenized reviews
    reviews2vectors.append(make_sent_vect(review))
reviews2vectors = np.array(reviews2vectors)

def most_similar_reviews(review):
    #Find the vector for this review
    v = make_sent_vect(review)
    most_similar = list()
    
    scores = Counter()
    #if two vectors are similar then their product (element by element and then sum) will be bigger
    for i,val in enumerate(reviews2vectors.dot(v)):
        scores[i] = val
    
    for idx,score in scores.most_common(3):
        most_similar.append(raw_reviews[idx])
    return most_similar

def most_similar_reviews_stavros(review):
    # Faster version, make matrix multiplication
    #Find the vector for this review
    v = make_sent_vect(review)
    v.resize(v.shape[0],1)    
    review_scores = np.dot(reviews2vectors, v)
    best3 = np.argsort(review_scores, axis=0)[::-1][:3]
    most_similar = list()
    for r in best3:
        most_similar.append(raw_reviews[int(r)])
    return most_similar

most_similar_reviews(['boring','awful'])

['no comment  stupid movie  acting average or worse . . . screenplay  no sense at all . . . skip it   \n',
 'i wouldn  t rent this one even on dollar rental night .  \n',
 'this video is     retarded . besides the brain cell killing acting and plot  it  s way too long . don  t waste your money at the video store . i actually was mad that i sat through this garbage and spent money on it . just absolutely awful .  \n']

In [130]:
%timeit most_similar_reviews(['boring','awful'])

18.4 ms ± 272 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
most_similar_reviews_stavros(['boring','awful'])

['no comment  stupid movie  acting average or worse . . . screenplay  no sense at all . . . skip it   \n',
 'i wouldn  t rent this one even on dollar rental night .  \n',
 'this video is     retarded . besides the brain cell killing acting and plot  it  s way too long . don  t waste your money at the video store . i actually was mad that i sat through this garbage and spent money on it . just absolutely awful .  \n']

In [131]:
%timeit most_similar_reviews_stavros(['boring','awful'])

4.1 ms ± 49.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Normalization

The point of normalization is to make variables comparable to each other.

In [38]:
norms[:1]

array([[0.34773861]])

In [47]:
#First you take each row and you sum its squared weights
np.sum(list(map(lambda x:x**2, weights_0_1[0:5,:])), axis=1)
# if you add keepdims=1 then there is no need for resize

array([[0.34773861],
       [0.4001928 ],
       [0.31854884],
       [0.42441024],
       [0.37752401]])

In [49]:
#norms.resize(norms.shape[0],1) creates a column matrix (instead of a row matrix)
norms[0:5]

array([0.34773861, 0.4001928 , 0.31854884, 0.42441024, 0.37752401])

In [37]:
# normed_weights is the weights multiplied (each column) with the normed_weights
normed_weights

array([[-0.00594589,  0.01537957, -0.03479716, ...,  0.00491532,
        -0.03442329,  0.00828448],
       [-0.01425374,  0.00224013,  0.0308414 , ..., -0.02168088,
         0.00961813,  0.03623475],
       [ 0.02865229,  0.00362072,  0.02647069, ..., -0.02681901,
         0.03079221, -0.02025997],
       ...,
       [-0.04057025, -0.03256796, -0.005784  , ..., -0.01269812,
        -0.00762553, -0.01685829],
       [-0.03169719, -0.02640913, -0.02081986, ..., -0.02510512,
        -0.00972922,  0.03498913],
       [ 0.01241721, -0.01713725,  0.00996594, ...,  0.00147966,
         0.00266909, -0.03177905]])

In [52]:
norms = np.sqrt(np.sum(weights_0_1 ** 2 ,axis=1, keepdims=1))
norms

array([[0.58969366],
       [0.63260794],
       [0.56440131],
       ...,
       [0.63350113],
       [0.60857029],
       [0.57917452]])

In [66]:
words=['this','is','a','good','movie']
make_sent_vect(words)

array([-0.1020982 , -0.06021745, -0.00360221, -0.09817285, -0.05933014,
       -0.01533265, -0.09012846, -0.06562662, -0.07816698, -0.01903082,
       -0.07730621, -0.05130723, -0.09841031, -0.09841245, -0.05194574,
       -0.01967773, -0.08983353, -0.03819163, -0.05328994,  0.01045728,
       -0.10898989, -0.05697306, -0.01559359, -0.02538482, -0.04711993,
       -0.06467783, -0.08864671, -0.03766169, -0.12726829, -0.04923117,
       -0.08250164, -0.09214022, -0.02291921, -0.00836783,  0.0204451 ,
       -0.08115479, -0.09363602, -0.09149977, -0.04685668, -0.092641  ,
       -0.0653623 , -0.13240103, -0.06144531, -0.08910579, -0.07003325,
       -0.0757983 , -0.0002148 , -0.11973187, -0.04660859, -0.04476614,
       -0.04897268, -0.02885444, -0.05793279,  0.00196865, -0.02640572,
       -0.00641995, -0.0517071 , -0.09895301, -0.06323607, -0.0128212 ,
       -0.02656843, -0.03967399, -0.10007042, -0.02573938, -0.10169405,
       -0.05913026, -0.01960393, -0.05321616, -0.02065232, -0.06

In [82]:
words=['this','is','a','good','movie']
this_review = make_sent_vect(words)
reviews2vectors[0].dot(this_review)

0.11457499442673733

In [73]:
np.dot(reviews2vectors, this_review.T)

array([0.11457499, 0.11035165, 0.07141358, ..., 0.10392294, 0.09140595,
       0.12656131])

In [83]:
reviews2vectors.shape

(25000, 100)

In [84]:
this_review.shape

(100,)

In [151]:
words=['boring','awful']
this_review = make_sent_vect(words)
this_review.resize(this_review.shape[0],1)
this_review.T

array([[ 0.11656979, -0.03717042,  0.04483297,  0.07379027,  0.11533581,
        -0.0532174 , -0.02377336, -0.02717042, -0.04595508,  0.04428797,
         0.12167114,  0.06485789,  0.11951096,  0.0059348 , -0.11142395,
         0.09682731,  0.10801687,  0.03978317,  0.07677372,  0.0285973 ,
         0.02481736, -0.0769564 , -0.06408165, -0.05550242, -0.04635753,
        -0.12897093,  0.0550261 , -0.01955263, -0.0079733 , -0.09731283,
         0.16769118, -0.08955328,  0.08544972, -0.09872328, -0.15019725,
         0.06836418, -0.0620496 ,  0.01191898, -0.0757797 , -0.01206828,
        -0.1270169 ,  0.06085481,  0.00281248, -0.10286752, -0.06072698,
         0.04018963,  0.05444234,  0.2092833 , -0.16261433, -0.09929475,
         0.08291038,  0.11714358, -0.02383672, -0.07581927,  0.00086753,
        -0.06742831, -0.20914505, -0.00261467, -0.10479984,  0.08466371,
         0.06961278, -0.16640474,  0.07871799,  0.06555903,  0.07629995,
        -0.14045947, -0.01900039, -0.1048364 ,  0.0

In [144]:
review_scores = np.dot(reviews2vectors, this_review)
best3 = np.argsort(review_scores, axis=0)[::-1][:3]
for r in best3:
    print(raw_reviews[int(r)])

no comment  stupid movie  acting average or worse . . . screenplay  no sense at all . . . skip it   

i wouldn  t rent this one even on dollar rental night .  

this video is     retarded . besides the brain cell killing acting and plot  it  s way too long . don  t waste your money at the video store . i actually was mad that i sat through this garbage and spent money on it . just absolutely awful .  



In [148]:
[raw_reviews[int(i)] for i in best3]

['no comment  stupid movie  acting average or worse . . . screenplay  no sense at all . . . skip it   \n',
 'i wouldn  t rent this one even on dollar rental night .  \n',
 'this video is     retarded . besides the brain cell killing acting and plot  it  s way too long . don  t waste your money at the video store . i actually was mad that i sat through this garbage and spent money on it . just absolutely awful .  \n']

In [149]:
[review_scores[int(i)] for i in best3]

[array([0.36046042]), array([0.35627966]), array([0.29942664])]

RNN

In [152]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('data/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [155]:
raw[:3]

['1 Mary moved to the bathroom.\n',
 '2 John went to the hallway.\n',
 '3 Where is Mary? \tbathroom\t1\n']

In [153]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)

vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
    
def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [154]:
len(vocab)

82

In [157]:
words2indices(['where','is','mary'])

[25, 56, 28]

In [184]:
np.random.seed(1)

#Embedding size = 10, that means that we wiil create a vector of 10 numbers for each word
embed_size = 20

# word embeddings
embed = (np.random.rand(len(vocab),embed_size) - 0.5) * 0.1

# embedding -> embedding (initially the identity matrix)
recurrent = np.eye(embed_size)

# sentence embedding for empty sentence
start = np.zeros(embed_size)

# embedding -> output weights
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1

# one hot lookups (for loss function)
one_hot = np.eye(len(vocab))

Forward Propagation with Arbitrary Length

In [203]:
def predict(sent, more_info=False):
    
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    # forward propagate
    #preds = list()
    for target_i in range(len(sent)):
        
        if more_info:
            print(f'Word: {vocab[sent[target_i]]}')
            
        layer = {}

        # try to predict the next term
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))

        loss += -np.log(layer['pred'][sent[target_i]])

        # generate the next hidden state
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)

    return layers, loss


Backpropagation with Arbitrary Length

In [186]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):  # if not the first layer
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

Weight Update with Arbitrary Length

In [187]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter % len(tokens)][1:])

    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a 
            # later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else:
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

    # update weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx,layer in enumerate(layers[1:]):
        
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))

Perplexity:81.90689914965836
Perplexity:81.54874909835365
Perplexity:81.03952457929005
Perplexity:80.14267876135821
Perplexity:78.31074152740617
Perplexity:73.76327560814543
Perplexity:56.13299305501793
Perplexity:28.223751127519517
Perplexity:20.12824121277819
Perplexity:18.539255904287455
Perplexity:17.076770581229606
Perplexity:14.986328527053107
Perplexity:11.994811346474723
Perplexity:8.85898998314854
Perplexity:7.141389277973937
Perplexity:6.202298539780477
Perplexity:5.538030602558332
Perplexity:5.123305561689193
Perplexity:4.843939793340768
Perplexity:4.647976745666142
Perplexity:4.517704396878363
Perplexity:4.445064840498462
Perplexity:4.39488698107356
Perplexity:4.333269320320349
Perplexity:4.255353662992695
Perplexity:4.165874552131964
Perplexity:4.071358453102025
Perplexity:3.9723242218998362
Perplexity:3.896626409647608
Perplexity:3.8757768816207445


Execution and Output Analysis

In [204]:
sent_index = 4

l,_ = predict(words2indices(tokens[sent_index]), True)

print(tokens[sent_index])

for i,each_layer in enumerate(l[1:-1]):
    input = tokens[sent_index][i]
    true = tokens[sent_index][i+1]
    pred = vocab[each_layer['pred'].argmax()]
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

Word: sandra
Word: moved
Word: to
Word: the
Word: garden.
['sandra', 'moved', 'to', 'the', 'garden.']
Prev Input:sandra      True:moved          Pred:is
Prev Input:moved       True:to             Pred:to
Prev Input:to          True:the            Pred:the
Prev Input:the         True:garden.        Pred:bedroom.


In [177]:
#words2indices(tokens[2 % len(tokens)][1:])
tokens[0 % len(tokens)][1:]

['moved', 'to', 'the', 'bathroom.']

In [189]:
embed[:2]

array([[ 0.24618294,  0.25083996, -0.05009852, -0.21215657, -0.21104805,
        -0.30466606, -0.5333955 , -0.23242285,  0.50067959, -0.06018655,
         0.08897665, -0.06850264,  0.00411577,  0.6148313 , -0.28214099,
        -0.4283181 , -0.30010674, -0.08342092,  0.06842135, -0.06700439],
       [ 0.02731105,  0.0433735 , -0.01456817,  0.02196471,  0.04397357,
         0.04522468, -0.04135362, -0.04734088, -0.04050605,  0.04174311,
        -0.03779046, -0.00585994,  0.04462329, -0.00376893,  0.02280841,
        -0.01337451,  0.02232304,  0.03404385, -0.04998701,  0.02538059]])

In [190]:
recurrent.shape

(20, 20)

In [191]:
decoder.shape

(20, 82)